In [3]:
import os
import chromadb
from dotenv import load_dotenv
load_dotenv()

VECTORDB_PATH = os.getenv('VECTORDB_PATH', './vectordb')
if not os.path.isabs(VECTORDB_PATH):
    VECTORDB_PATH = os.path.abspath(VECTORDB_PATH)

client = chromadb.PersistentClient(path=VECTORDB_PATH)

# 컬렉션 리스트 조회
def list_collections():
    collections = [c.name for c in client.list_collections()]
    print(f"[INFO] 현재 벡터DB에 존재하는 컬렉션: {collections}")
    return collections

# 각 컬렉션에서 데이터 일부 샘플 조회
def show_sample_from_collection(collection_name, n=3):
    if collection_name not in [c.name for c in client.list_collections()]:
        print(f"[WARN] 컬렉션 '{collection_name}'이 존재하지 않습니다.")
        return
    collection = client.get_collection(collection_name)
    results = collection.get()
    print(f"[INFO] '{collection_name}' 내 데이터 개수: {len(results['ids'])}")
    for i, (doc, meta) in enumerate(zip(results['documents'], results['metadatas'])):
        if i >= n:
            break
        print(f"[{i+1}] doc: {doc}\n    meta: {meta}")


In [4]:
collections = list_collections()
# 각 컬렉션에서 샘플 데이터 3개씩 출력
for cname in collections:
    show_sample_from_collection(cname, n=3) 

[INFO] 현재 벡터DB에 존재하는 컬렉션: ['ldl_cholesterol', 'hdl_cholesterol', 'blood_glucose', 'spo2', 'blood_pressure']
[INFO] 'ldl_cholesterol' 내 데이터 개수: 9
[1] doc: {'total_count': 9, 'id': 180, 'memb_id': '52a62c11-7c4f-4912-91c9-ff5c145328cd', 'device_desc': None, 'fullname': '김테스터', 'get_time': '2024-03-09 20:24:40', 'cho_value': 42, 'create_dttm': '2024-03-09 20:26:55'}
    meta: {'data_type': 'ldl_cholesterol', 'device_desc': '', 'created_by_system': 'FaMiliCare', 'create_dttm': '2024-03-09 20:26:55', 'ingest_time': '2025-07-21 15:51:55', 'is_private': True, 'anonymized': False, 'fullname': '김테스터', 'data_source': '', 'consent_granted': True, 'memb_id': '52a62c11-7c4f-4912-91c9-ff5c145328cd', 'get_time': '2024-03-09 20:24:40'}
[2] doc: {'total_count': 9, 'id': 176, 'memb_id': '52a62c11-7c4f-4912-91c9-ff5c145328cd', 'device_desc': None, 'fullname': '김테스터', 'get_time': '2024-03-06 15:31:47', 'cho_value': 42, 'create_dttm': '2024-03-06 15:33:59'}
    meta: {'device_desc': '', 'memb_id': '52a62c1

In [5]:
collection = client.get_collection("blood_glucose")

In [6]:
count = collection.count()
print("blood_glucose 컬렉션의 전체 데이터 건수:", count)

blood_glucose 컬렉션의 전체 데이터 건수: 600


In [7]:
# 일부 데이터(예: 3개) 조회 - 임베딩과 메타데이터 포함
docs = collection.get(
    include=["embeddings", "metadatas", "documents"],  # 포함할 항목
    limit=1
)
docs

{'ids': ['4906'],
 'embeddings': array([[ 1.02867275e-01, -2.87042797e-01, -5.67302220e-02,
         -2.99808234e-01,  2.08355829e-01, -4.12890345e-01,
          2.12759599e-01,  6.95273504e-02, -1.02512807e-01,
         -5.13478756e-01,  2.78424889e-01, -1.75879285e-01,
         -1.64557323e-01,  4.70974207e-01, -8.34069908e-01,
          1.59713730e-01, -4.71102685e-01,  1.87951490e-01,
          2.47088060e-01, -3.20891351e-01, -4.52915393e-02,
          2.35764310e-01,  5.63564241e-01,  4.16953415e-01,
         -4.08505470e-01,  6.73441470e-01,  3.66648912e-01,
         -5.04613876e-01,  4.52154398e-01,  1.54138086e-02,
         -6.35175467e-01, -3.04262370e-01, -2.59197980e-01,
          8.26691464e-02,  9.20229256e-01, -3.15323472e-01,
         -6.04110956e-01,  5.99626064e-01, -4.48622435e-01,
         -2.45760560e-01, -4.28107828e-01,  3.66199017e-01,
          2.19822407e-01,  5.74752867e-01, -4.78789844e-02,
         -1.01636559e-01, -1.42127320e-01, -1.44273536e-02,
        

In [18]:
# 일부 데이터(예: 3개) 조회 - 임베딩과 메타데이터 포함
docs = collection.get(
    include=["embeddings", "metadatas", "documents"],  # 포함할 항목
    limit=3
)

# 결과 출력
for i in range(len(docs['ids'])):
    print(f"[{i}] ID: {docs['ids'][i]}")
    print(f"문서 내용: {docs['documents'][i]}")
    print(f"메타데이터: {docs['metadatas'][i]}")
    print(f"임베딩 벡터 (길이 {len(docs['embeddings'][i])}): {docs['embeddings'][i][:5]}...")  # 일부만 출력
    print("-" * 50)

[0] ID: 4906
문서 내용: {'total_count': 599, 'id': 4906, 'memb_id': '52a62c11-7c4f-4912-91c9-ff5c145328cd', 'device_desc': '혈당계(N Premier)', 'fullname': '김테스터', 'get_time': '2025-03-01 02:12:42', 'glucosedata': 107, 'when_eat': '식사 후 4시간', 'create_dttm': '2025-03-01 02:12:59'}
메타데이터: {'when_eat': '식사 후 4시간', 'device_desc': '혈당계(N Premier)', 'get_time': '2025-03-01 02:12:42', 'anonymized': False, 'data_type': 'blood_glucose', 'fullname': '김테스터', 'ingest_time': '2025-07-21 14:27:16', 'is_private': True, 'created_by_system': 'FaMiliCare', 'data_source': '혈당계(N Premier)', 'memb_id': '52a62c11-7c4f-4912-91c9-ff5c145328cd', 'consent_granted': True, 'create_dttm': '2025-03-01 02:12:59'}
임베딩 벡터 (길이 768): [ 0.1028674  -0.28704309 -0.05673055 -0.29980794  0.20835578]...
--------------------------------------------------
[1] ID: 4905
문서 내용: {'total_count': 599, 'id': 4905, 'memb_id': '52a62c11-7c4f-4912-91c9-ff5c145328cd', 'device_desc': '혈당계(N Premier)', 'fullname': '김테스터', 'get_time': '2025-02-24 0